In [1]:
#====================================================================================
#    Proof-of-concept SageMath 9.2 implementation of the algorithms
#    proposed in "Entropoid Based Cryptography"
#
#    Copyright 2020, 2021:
#    Danilo Gligoroski, <danilog@ntnu.no>
#    Department of Information Security and Communication Technology,
#    Faculty of Information Technology and Electrical Engineering,
#    Norwegian University of Science and Technology - NTNU,
#    O.S.Bragstads plass 2B, 
#    N-7034 Trondheim, Norway
#
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
#
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
#
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <https://www.gnu.org/licenses/>.
#====================================================================================

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm
from numpy import ma
from matplotlib import ticker, cm
from sage.plot.contour_plot import ContourPlot

import re
import scipy
import scipy.stats

import random

# Finds a prime p = 2 q + 1 such that q is the the smallest prime 
# larger than 2^(lambda_bits - 2) + 2^(lambda_bits - 3)
def get_fixed_safe_prime(nbits = 128):
    # nbits should be >= 3
    global p, q
    q = next_prime(2^(nbits-2) + 2^(nbits - 3))
    search = True
    while search:
        p = 2*q + 1
        search = not(is_prime(p))
        q = next_prime(q)
    return p

def get_random_safe_prime(nbits = 128):
    # nbits should be >= 3
    global p, q
    q = random_prime(2^(nbits-2), false, 2^(nbits - 3))
    search = True
    while search:
        p = 2*q + 1
        search = not(is_prime(p))
        q = next_prime(q)
    return p

def get_random_entropoid_coefficients():
    global aa1, aa3, aa4, aa8, bb1, bb2, bb5, bb7, E_Zero, E_left_unit
    global F, p
    aa3 = F.random_element()
    aa8 = 0
    while aa8 == 0:
        aa8 = F.random_element(); 
    bb2 = F.random_element()
    bb7 = 0
    while bb7 == 0:
        bb7 = F.random_element(); 

    aa1 = (aa3*(aa8*bb2 - bb7))/(aa8*bb7)
    aa4 = (aa8*bb2)/bb7
    bb1 = -((bb2*(aa8 - aa3*bb7))/(aa8*bb7))
    bb5 = (aa3*bb7)/aa8

    E_Zero = vector((-(aa3/aa8), -(bb2/bb7)))
    E_left_unit = vector((-(aa3/aa8) + 1/bb7, 1/aa8 - bb2/bb7))
    
    pass

def mmult( X, Y ):
    global aa1, aa3, aa4, aa8, bb1, bb2, bb5, bb7
    f1 = aa8*X[1]*Y[0] + aa3*X[1] + aa4*Y[0] + aa1;
    f2 = bb7*X[0]*Y[1] + bb2*X[0] + bb5*Y[1] + bb1;
    return(vector((f1, f2)))

def pplus(x, y):
    global E_Zero
    return(vector(x)+vector(y)-vector(E_Zero))

def mminus(x, y):
    global E_Zero
    return(vector(x)-vector(y)+vector(E_Zero))

def iinverse(x):
    global aa3, bb2, bb7, aa8
    return(vector(((1 - aa3 * bb2 - aa3 * bb7 * x[1])/(aa8 * (bb2 + bb7 * x[1])), \
           (1 - aa3 * bb2 - aa8 * bb2 * x[0])/(bb7 * (aa3 + aa8 * x[0])))))

def Narayana_sequence(base):
    seq = [binomial(base - 1, i) * binomial(base - 1, i - 1)/(base - 1) for i in range(1, base)]
    return(seq)

def Catalan_number(b):
    return((binomial(2*b, b))/(b+1))

def get_base_max(p):
    maxvalue = (p-1)^2
    bmax = 1
    while Catalan_number(bmax) < maxvalue:
        bmax += 1
    return(bmax)

def get_all_bracketing(n, base):
    nndigits = n.digits(base-1)
    nrdigits = len(nndigits)
    maxvalue = ZZ([base - 2 for i in range(nrdigits-1)], base-1)
    PPall = [ZZ(i).digits(base-1, padto=nrdigits-1) for i in range(maxvalue+1)]
    return(PPall)

# A procedure that given an integer n, and a base b
# produces an array of integers with values between 0 and b - 2 (including b - 2)
# The length of the array is the same as the length of n interpreted in the base base
def get_random_bracketing(n, base):
    bracket_pattern = [ random.randint(0, base - 2) for i in range(len(n.digits(base))) ]
    return(bracket_pattern)

# A procedure that produces a random power index
# The range of a is between 1 and p - 1
# Once a is obtained, the bracketing is obtained by calling a procedure 
# for random bracketing a in the used base.
def get_random_power_index(base):
    global p
    a = ZZ(random.randint(1, p))
    a_pattern = get_random_bracketing(a, base)
    return([a, a_pattern, base])

# If we want to generate the bracketing sapes we can use this procedure
def allassocrules(n):
    rules = ['mmult(gg,gg)']
    for i in range(n-2):
        qq = rules[0]
        tmp = 'mmult(' + qq + ',gg)'
        for j in range(len(rules)):
            rules[j] = 'mmult(gg,' + rules[j] + ')'
        rules.append(tmp)
    return (rules)

# Let us generate all basic bracketing shapes as defined in
# Definition 17 of the paper "Entropoid Based Cryptography"
limitbases = 258
AssocRulesForDifferentBases = [[] for i in range(limitbases)]
for i in range(2, limitbases):
    AssocRulesForDifferentBases[i] = allassocrules(i)

# This is a computational procedure that gives the same result as 
# we would have evaluated the symbolic bracketing AssocRulesForDifferentBases[base][digit]
def AssocRuleForBaseAndDigit(gg, base, digit):
    j = 0
    tmp = mmult(gg, gg)
    if digit == 0:
        while j < base - 2:
            tmp = mmult(gg, tmp)
            j += 1
        return(tmp)
    else:
        while j < digit - 1:
            tmp = mmult(gg, tmp)
            j += 1
        if j <= base - 2:
            tmp = mmult(tmp, gg)
            j += 1
        while j < base - 2:
            tmp = mmult(gg, tmp)
            j += 1
        return(tmp)


# This is the central and most important procedure for
# non-abelian and non-associative exponentiation of g 
# to the power index nn = [n, n_pattern, base] 
def nanapow(g, nn):
    n = nn[0]
    pattern = nn[1]
    base = nn[2]

    digitss = n.digits(base)

    # Compute equations (39) from the paper
    gg = g
    powlist = [gg]
    for i in range(1, len(pattern)):
        gg = AssocRuleForBaseAndDigit(gg, base, pattern[i])
        powlist.append(gg)

    # find the index of the first non-zero element
    ii = digitss.index(next(filter(lambda x: x!=0, digitss))) 
    
    # Compute equations (40) from the paper
    if digitss[ii] == 1:
        result = powlist[ii]
    else:
        gg = powlist[ii]
        result = AssocRuleForBaseAndDigit(gg, digitss[ii], pattern[ii]%(digitss[ii] - 1))

    # Compute equations (41) from the paper
    ii += 1
    while ii<len(digitss):
        if digitss[ii] != 0:
            if digitss[ii] == 1:
                tmp = powlist[ii]
            else:
                gg = powlist[ii]
                tmp = AssocRuleForBaseAndDigit(gg, digitss[ii], pattern[ii]%(digitss[ii] - 1))
            if pattern[ii-1]%2 == 0:
                result = mmult(tmp, result)                
            else:
                result = mmult(result, tmp)
        ii += 1
    
    return(result)

# A procedure that finds a generator of the multiplicative quasigroup (\mathbb{E}_{(p-1)^2}^*, *)
def find_generator_p():
    global F, p
    base = 2
    search = True
    trials = 0
    while search:
        trials += 1
        Success = True
        g = vector((F.random_element(), F.random_element()))
        Success = Success and (g != nanapow(g, [ZZ(p), get_random_bracketing(ZZ(p), base), base]))
        tmp2 = mmult(g, g)
        Success = Success and (tmp2 != nanapow(g, [ZZ(p-1), get_random_bracketing(ZZ(p), base), base]))
        tmp3  = mmult(g, tmp2)
        Success = Success and (tmp3 != nanapow(g, [ZZ(p-2), get_random_bracketing(ZZ(p), base), base]))
        Success = Success and (tmp3 != mmult(tmp2, g))
        Success = Success and (mmult(g, tmp3) != mmult(tmp3, g))
        search = not(Success)
    return(g)

# A procedure that finds a generator of the Sylow q-subquasigroup (\mathbb{E}_{q^2}^*, *)
def find_generator_q():
    global F, p
    g = find_generator_p()
    return( mmult(g,mmult(g,mmult(g,mmult(mmult(g,g),g)))) )

def GenParameters_Diffie_Hellman(lambda_bits):
    global aa1, aa3, aa4, aa8, bb1, bb2, bb5, bb7, E_Zero, E_left_unit
    global F, p, g, g_q
    
    p = get_fixed_safe_prime(lambda_bits) # or get_random_safe_prime(lambda_bits)
    print("p = ", p)
    print("log_2(p) = ", ceil(log(p, 2)))
    print()
    
    F = GF(p)
    print("Information about the finite field: ", F)
    print()

    # Get some appropriate but random coefficients that define E_{p^2}
    get_random_entropoid_coefficients()
    print("aa1 = ", aa1)
    print("aa3 = ", aa3)
    print("aa4 = ", aa4)
    print("aa8 = ", aa8)
    print("bb1 = ", bb1)
    print("bb2 = ", bb2)
    print("bb5 = ", bb5)
    print("bb7 = ", bb7)
    print()

    # Some of the characteristic elements in the Entropoid 
    print("E_Zero = ", E_Zero)
    print("E_left_unit = ", E_left_unit)
    E_minus_one = mminus(E_Zero, E_left_unit)
    print("E_minus_one = ", E_minus_one)
    print()

    # g is a generator for the multiplicative quasigroup of E_{p^2}
    g = find_generator_p()
    print("g = ", g)
    print()

    # g_q is a generator for the Silow q subquasigroup 
    g_q = find_generator_q()
    print("g_q = ", g_q)
    print()

import binascii
import hashlib
def concatenate_entropoid_element_and_message(I, M, I_size_in_bytes):
    byte_array = b''
    for i in range(len(I)):
        byte_array = byte_array + int(I[i]).to_bytes(I_size_in_bytes, 'little')
    byte_array = byte_array + bytearray(M.encode())
    return byte_array

# Hash to power indices procedures for Signatures01
def Hash512_to_power_index(byte_array, base):
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    ss = hashlib.sha512(byte_array).hexdigest()
    ss_first_half = ss[:64]; 
    bb1 = ZZ(int('0x'+ss_first_half,16))
    bb1_in_base = bb1.digits(base); 
    ss_second_half = ss[64:]; 
    bb2 = ZZ(int('0x'+ss_second_half,16))
    bb2_in_base_min_1 = bb2.digits(base - 1)
    bb2_in_base_min_1 = bb2_in_base_min_1[:len(bb1_in_base)]
    return([bb1, bb2_in_base_min_1, base])

def Hash384_to_power_index(byte_array, base):
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    ss = hashlib.sha384(byte_array).hexdigest()
    ss_first_half = ss[:48]; 
    bb1 = ZZ(int('0x'+ss_first_half,16))
    bb1_in_base = bb1.digits(base); 
    ss_second_half = ss[48:]; 
    bb2 = ZZ(int('0x'+ss_second_half,16))
    bb2_in_base_min_1 = bb2.digits(base - 1)
    bb2_in_base_min_1 = bb2_in_base_min_1[:len(bb1_in_base)]
    return([bb1, bb2_in_base_min_1, base])

def Hash256_to_power_index(byte_array, base):
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    ss = hashlib.sha256(byte_array).hexdigest()
    ss_first_half = ss[:32]; 
    bb1 = ZZ(int('0x'+ss_first_half,16))
    bb1_in_base = bb1.digits(base); 
    ss_second_half = ss[32:]; 
    bb2 = ZZ(int('0x'+ss_second_half,16))
    bb2_in_base_min_1 = bb2.digits(base - 1)
    bb2_in_base_min_1 = bb2_in_base_min_1[:len(bb1_in_base)]
    return([bb1, bb2_in_base_min_1, base])

def GenKey_Signature01(lambda_bits, base):
    global aa1, aa3, aa4, aa8, bb1, bb2, bb5, bb7, E_Zero, E_left_unit
    global F, p, g, g_q, B, PrivateKey, PublicKey, x, y
    
    if not(lambda_bits in {128, 192, 256}):
        print("ERROR: lambda_bits should be either 128 or 192 or 256")
        return(-1)
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    
    p = get_fixed_safe_prime(lambda_bits) # or get_random_safe_prime(lambda_bits)
    print("p = ", p)
    print("log_2(p) = ", ceil(log(p, 2)))
    print()
    
    F = GF(p)
    print("Information about the finite field: ", F)
    print()

    # Get some appropriate but random coefficients that define E_p
    get_random_entropoid_coefficients()
    print("aa1 = ", aa1)
    print("aa3 = ", aa3)
    print("aa4 = ", aa4)
    print("aa8 = ", aa8)
    print("bb1 = ", bb1)
    print("bb2 = ", bb2)
    print("bb5 = ", bb5)
    print("bb7 = ", bb7)
    print()

    # Some of the characteristic elements in the Entropoid 
    print("E_Zero = ", E_Zero)
    print("E_left_unit = ", E_left_unit)
    E_minus_one = mminus(E_Zero, E_left_unit)
    print("E_minus_one = ", E_minus_one)
    print()

    # g is a generator for the multiplicative quasigroup of E_p
    g = find_generator_p()
    print("g = ", g)
    print()

    # g_q is a generator for the Silow q subquasigroup 
    g_q = find_generator_q()
    print("g_q = ", g_q)
    print()

    # generate a random public parameter B
    # B is a power index
    M = "This is a seed message for fixing the value of the public root."
    if lambda_bits == 128:
        B = Hash256_to_power_index(bytearray(M.encode()), base)
    if lambda_bits == 192:
        B = Hash384_to_power_index(bytearray(M.encode()), base)
    if lambda_bits == 256:
        B = Hash512_to_power_index(bytearray(M.encode()), base)

    print("Public power index B = ", B)
    print()

    # generate the PrivateKey x
    # skip the coordinates of E_Zero
    x0 = E_Zero[0]
    while x0 == E_Zero[0]:
        x0 = F.random_element(); 
    x1 = E_Zero[1]
    while x1 == E_Zero[1]:
        x1 = F.random_element(); 
    x = vector((x0, x1))
    PrivateKey = x
    print("PrivateKey = ", PrivateKey)
    print()
    
    # generate the PublicKey y
    y = nanapow(x, B)
    PublicKey = y
    print("PublicKey = ", PublicKey)
    print()

def Sign_Signature01(PrivateKey, Message, base, lambda_bits):
    global B
    # generate a random r
    # skip the coordinates of E_Zero
    r0 = E_Zero[0]
    while r0 == E_Zero[0]:
        r0 = F.random_element(); 
    r1 = E_Zero[1]
    while r1 == E_Zero[1]:
        r1 = F.random_element(); 
    r = vector((r0, r1))
    
    I = nanapow(r, B)
    I_and_Message = concatenate_entropoid_element_and_message(I, Message, lambda_bits//8)
    
    if lambda_bits == 128:
        H = Hash256_to_power_index(I_and_Message, base)
    if lambda_bits == 192:
        H = Hash384_to_power_index(I_and_Message, base)
    if lambda_bits == 256:
        H = Hash512_to_power_index(I_and_Message, base)
        
    s = nanapow(mmult(PrivateKey, r), H)
    
    signature = (I, s)
    return signature

def Verify_Signature01(PublicKey, Message, signature, base, lambda_bits):
    global B
    
    I = signature[0]
    I_and_Message = concatenate_entropoid_element_and_message(I, Message, lambda_bits//8)
    
    if lambda_bits == 128:
        H = Hash256_to_power_index(I_and_Message, base)
    if lambda_bits == 192:
        H = Hash384_to_power_index(I_and_Message, base)
    if lambda_bits == 256:
        H = Hash512_to_power_index(I_and_Message, base)
        
    s_to_B = nanapow(signature[1], B)
    y_mult_I_to_H = nanapow(mmult(PublicKey, I), H)
    
    if s_to_B == y_mult_I_to_H:
        return True
    else:
        return False

# Hash to power indices for Signatures02
def Hash512_to_power_index_02(byte_array, base):
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    ss = hashlib.sha512(byte_array).hexdigest()
    bb_s = ZZ(int('0x'+ss,16))
    bb_s_in_base_min_1 = bb_s.digits(base - 1)
    bb_s_in_base_min_1 = bb_s_in_base_min_1[:len(q.digits(base))]
    return([q, bb_s_in_base_min_1, base])

def Hash384_to_power_index_02(byte_array, base):
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    ss = hashlib.sha384(byte_array).hexdigest()
    bb_s = ZZ(int('0x'+ss,16))
    bb_s_in_base_min_1 = bb_s.digits(base - 1)
    bb_s_in_base_min_1 = bb_s_in_base_min_1[:len(q.digits(base))]
    return([q, bb_s_in_base_min_1, base])

def Hash256_to_power_index_02(byte_array, base):
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    ss = hashlib.sha256(byte_array).hexdigest()
    bb_s = ZZ(int('0x'+ss,16))
    bb_s_in_base_min_1 = bb_s.digits(base - 1)
    bb_s_in_base_min_1 = bb_s_in_base_min_1[:len(q.digits(base))]
    return([q, bb_s_in_base_min_1, base])

def GenKey_Signature02(lambda_bits, base):
    global aa1, aa3, aa4, aa8, bb1, bb2, bb5, bb7, E_Zero, E_left_unit
    global F, q, p, g, g_q, B, PrivateKey, PublicKey, x, y
    
    if not(lambda_bits in {256, 384, 512}):
        print("ERROR: lambda_bits should be either 128 or 192 or 256")
        return(-1)
    if not(base in {17, 257}):
        print("ERROR: Base should be either 17 or 257")
        return(-2)
    
    p = get_fixed_safe_prime(lambda_bits) # or get_random_safe_prime(lambda_bits)
    print("p = ", p)
    print("log_2(p) = ", ceil(log(p, 2)))
    print()
    print("q = ", q)
    print("log_2(q) = ", ceil(log(q, 2)))
    print()
    
    F = GF(p)
    print("Information about the finite field: ", F)
    print()

    # Get some appropriate but random coefficients that define E_{p^2}
    get_random_entropoid_coefficients()
    print("aa1 = ", aa1)
    print("aa3 = ", aa3)
    print("aa4 = ", aa4)
    print("aa8 = ", aa8)
    print("bb1 = ", bb1)
    print("bb2 = ", bb2)
    print("bb5 = ", bb5)
    print("bb7 = ", bb7)
    print()

    # Some of the characteristic elements in the Entropoid 
    print("E_Zero = ", E_Zero)
    print("E_left_unit = ", E_left_unit)
    E_minus_one = mminus(E_Zero, E_left_unit)
    print("E_minus_one = ", E_minus_one)
    print()

    # g is a generator for the multiplicative quasigroup of E_{p^2}
    g = find_generator_p()
    print("g = ", g)
    print()

    # g_q is a generator for the Silow q subquasigroup 
    g_q = find_generator_q()
    print("g_q = ", g_q)
    print()

    # generate a random public parameter B
    # B is a power index
    M = "This is a seed message for fixing the value of the public root."
    if lambda_bits == 256:
        B = Hash256_to_power_index_02(bytearray(M.encode()), base)
    if lambda_bits == 384:
        B = Hash384_to_power_index_02(bytearray(M.encode()), base)
    if lambda_bits == 512:
        B = Hash512_to_power_index_02(bytearray(M.encode()), base)

    print("Public power index B = ", B)
    print()

    # generate the PrivateKey x
    # skip the coordinates of E_Zero
    x0 = E_Zero[0]
    while x0 == E_Zero[0]:
        x0 = F.random_element(); 
    x1 = E_Zero[1]
    while x1 == E_Zero[1]:
        x1 = F.random_element(); 
    x = vector((x0, x1))
    PrivateKey = x
    print("PrivateKey = ", PrivateKey)
    print()
    
    # generate the PublicKey y
    y = nanapow(x, B)
    PublicKey = y
    print("PublicKey = ", PublicKey)
    print()

def Sign_Signature02(PrivateKey, Message, base, lambda_bits):
    global B
    # generate a random r
    # skip the coordinates of E_Zero
    r0 = E_Zero[0]
    while r0 == E_Zero[0]:
        r0 = F.random_element(); 
    r1 = E_Zero[1]
    while r1 == E_Zero[1]:
        r1 = F.random_element(); 
    r = vector((r0, r1))
    
    I = nanapow(r, B)
    I_and_Message = concatenate_entropoid_element_and_message(I, Message, lambda_bits//8)
    
    if lambda_bits == 256:
        H = Hash256_to_power_index_02(I_and_Message, base)
    if lambda_bits == 384:
        H = Hash384_to_power_index_02(I_and_Message, base)
    if lambda_bits == 512:
        H = Hash512_to_power_index_02(I_and_Message, base)
        
    s = nanapow(mmult(PrivateKey, r), H)
    
    signature = (I, s)
    return signature

def Verify_Signature02(PublicKey, Message, signature, base, lambda_bits):
    global B
    
    I = signature[0]
    I_and_Message = concatenate_entropoid_element_and_message(I, Message, lambda_bits//8)
    
    if lambda_bits == 256:
        H = Hash256_to_power_index_02(I_and_Message, base)
    if lambda_bits == 384:
        H = Hash384_to_power_index_02(I_and_Message, base)
    if lambda_bits == 512:
        H = Hash512_to_power_index_02(I_and_Message, base)
        
    s_to_B = nanapow(signature[1], B)
    y_mult_I_to_H = nanapow(mmult(PublicKey, I), H)
    
    if s_to_B == y_mult_I_to_H:
        return True
    else:
        return False

# Experiments with entropoid Diffie-Hellman key exchange

In [3]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

# Set lambda_bits for a safe prime p = 2 q + 1. 
# Notice that the prime number q will have lambda_bits - 1 bits
lambda_bits = 128
print("lambda_bits = ", lambda_bits)
print()

GenParameters_Diffie_Hellman(lambda_bits)

initial seed:  0
lambda_bits =  128

p =  255211775190703847597530955573826162347
log_2(p) =  128

Information about the finite field:  Finite Field of size 255211775190703847597530955573826162347

aa1 =  76196257025066871002891471619848650286
aa3 =  222398418873342498392506793217273317643
aa4 =  241322191999219874657044597267756778501
aa8 =  178720374391503507560897272085794393770
bb1 =  47717732877487248530043326597560953714
bb2 =  61733507447498311892063876037630235726
bb5 =  247856823686916680700123131434371243080
bb7 =  27228823810010487136687635290782267614

E_Zero =  (25958627203386039978290716591117649049, 211647247068710178442880791600586184585)
E_left_unit =  (174395086300108117424968928287131469871, 150946712368694434305233963210010197064)
E_minus_one =  (132733943297367810129143460468929990574, 17136006578022074982996664417336009759)

g =  (13486814435588948056647180649177578889, 232946345282352956340474695993456704856)

g_q =  (128137692344645973157570713635711768143, 2002

In [4]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())
print()

# A simmulation of a Diffie-Hellman key exchange with g

# First test: Alice and Bob know E_p, g, and work with a same base

# From the Section 2.4 Dichotomy between odd and even bases
# in the paper, the recommended values are odd bases
base = 17

base_alice = base
base_bob = base
print("Agreed common base = ", base)
print()

# Alice gets a random power index
Alice = get_random_power_index(base_alice); 
print("Alice = ", Alice)
# Alice produces her public part Ka by calling the procedure nanapow() 
# that uses two parameters g and the power index Alice in order to 
# calculate g^Alice
Ka = nanapow(g, Alice)
print("Ka = ", Ka)
print()

# Bob gets a random power index
Bob = get_random_power_index(base_bob); 
print("Bob = ", Bob)
# Bob produces his public part Kb by calling the procedure nanapow() 
# that uses two parameters g and the power index Bob in order to 
# calculate g^Bob
Kb = nanapow(g, Bob)
print("Kb = ", Kb)
print()

# After receiving Kb, Alice computes the shared key Kab
Kab = nanapow(Kb, Alice)
print("Kab = ", Kab)
print()

# After receiving Ka, Bob computes the shared key Kba
Kba = nanapow(Ka, Bob)
print("Kba = ", Kba)
print()

print("The shared keys are the same: ", Kab == Kba)

initial seed:  0

Agreed common base =  17

Alice =  [47392387440050222334387056025351624212, [9, 4, 3, 8, 4, 9, 3, 2, 10, 15, 3, 11, 13, 10, 6, 15, 14, 8, 1, 0, 2, 12, 0, 15, 10, 7, 10, 2, 6, 7, 7], 17]
Ka =  (120421544237529629217788738702892637460, 127399315981183519023472362443200305215)

Bob =  [27374292381938936809021261532918539202, [10, 15, 3, 9, 9, 3, 10, 6, 9, 14, 2, 12, 10, 7, 9, 5, 6, 5, 1, 8, 15, 2, 2, 4, 4, 1, 2, 12, 8, 7, 6], 17]
Kb =  (75026280824958693170939039055689469463, 160666520418747394653273412070345743677)

Kab =  (125453479868602117491962117002259721097, 80231683823004324074180727294360287190)

Kba =  (125453479868602117491962117002259721097, 80231683823004324074180727294360287190)

The shared keys are the same:  True


In [5]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())
print()

# A simmulation of a Diffie-Hellman key exchange with g

# Second test: Alice and Bob know E_p, g, and work with bases on their own choice

base_alice = 9
print("base_alice = ", base_alice)
# Alice gets a random power index
Alice = get_random_power_index(base_alice); 
print("Alice = ", Alice)
# Alice produces her public part Ka by calling the procedure nanapow() 
# that uses two parameters g and the power index Alice in order to 
# calculate g  to  the power of Alice
Ka = nanapow(g, Alice)
print("Ka = ", Ka)
print()

base_bob = 33
print("base_bob = ", base_bob)
# Bob gets a random power index
Bob = get_random_power_index(base_bob); 
print("Bob = ", Bob)
# Bob produces his public part Kb by calling the procedure nanapow() 
# that uses two parameters g and the power index Bob in order to 
# calculate g  to the power of Bob
Kb = nanapow(g, Bob)
print("Kb = ", Kb)
print()

# After receiving Kb, Alice computes the shared key Kab
Kab = nanapow(Kb, Alice)
print("Kab = ", Kab)
print()

# After receiving Ka, Bob computes the shared key Kba
Kba = nanapow(Ka, Bob)
print("Kba = ", Kba)
print()

print("The shared keys are the same: ", Kab == Kba)


initial seed:  0

base_alice =  9
Alice =  [47392387440050222334387056025351624212, [4, 2, 1, 4, 2, 4, 1, 1, 5, 7, 1, 5, 6, 5, 3, 7, 7, 4, 0, 0, 1, 6, 0, 7, 5, 3, 5, 1, 3, 3, 3, 2, 7, 1, 1, 5, 7, 1, 4, 4], 9]
Ka =  (187978293522818904347073968325418320167, 4031884079618652321753082809363682574)

base_bob =  33
Bob =  [113229132117011277211194849153838560638, [13, 18, 28, 5, 24, 20, 15, 18, 11, 12, 11, 2, 16, 30, 4, 5, 8, 9, 2, 5, 25, 17, 15, 13, 26, 17], 33]
Kb =  (215810056254095917586510141651352944519, 58026588231455569467905033989562444838)

Kab =  (180300066404931116932038460669304112827, 140824513012217402735374548102038516194)

Kba =  (180300066404931116932038460669304112827, 140824513012217402735374548102038516194)

The shared keys are the same:  True


In [6]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

# Test with 192-bit prime numbers
lambda_bits = 192
print("lambda_bits = ", lambda_bits)
print()

GenParameters_Diffie_Hellman(lambda_bits)
print()

base_alice = 19
print("base_alice = ", base_alice)
# Alice gets a random power index
Alice = get_random_power_index(base_alice); 
print("Alice = ", Alice)
# Alice produces her public part Ka by calling the procedure nanapow() 
# that uses two parameters g and the power index Alice in order to 
# calculate g  to  the power of Alice
Ka = nanapow(g, Alice)
print("Ka = ", Ka)
print()

base_bob = 65
print("base_bob = ", base_bob)
# Bob gets a random power index
Bob = get_random_power_index(base_bob); 
print("Bob = ", Bob)
# Bob produces his public part Kb by calling the procedure nanapow() 
# that uses two parameters g and the power index Bob in order to 
# calculate g  to the power of Bob
Kb = nanapow(g, Bob)
print("Kb = ", Kb)
print()

# After receiving Kb, Alice computes the shared key Kab
Kab = nanapow(Kb, Alice)
print("Kab = ", Kab)
print()

# After receiving Ka, Bob computes the shared key Kba
Kba = nanapow(Ka, Bob)
print("Kba = ", Kba)
print()

print("The shared keys are the same: ", Kab == Kba)


initial seed:  0
lambda_bits =  192

p =  4707826301540010572876842067405749812076766583348025913783
log_2(p) =  192

Information about the finite field:  Finite Field of size 4707826301540010572876842067405749812076766583348025913783

aa1 =  257405322579552167145111504184189350941678348657606363000
aa3 =  119532791761269064716056035369728457738265053964404214027
aa4 =  739437136086943117194679493097867279546663219437537419926
aa8 =  3296809007157619640119308770869998958606195085103878511269
bb1 =  1271238259397074869634643961725054659706124524993994915864
bb2 =  2806677724762809107594599379975401118512734879194091885646
bb5 =  4599163888442026094155062442360406257983778255665643166463
bb7 =  949464039987904334513186654173522437107012951521210331273

E_Zero =  (2674838327407532648726594500168436242421616629939967512484, 1660530161738285992366893199821047810162372675981459243468)
E_left_unit =  (3665473625056202058790254267668269814529715265285076709145, 27221009023058919651243937394975

In [7]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

# Test with 256-bit prime numbers
lambda_bits = 256
print("lambda_bits = ", lambda_bits)
print()

GenParameters_Diffie_Hellman(lambda_bits)
print()

base_alice = 19
print("base_alice = ", base_alice)
# Alice gets a random power index
Alice = get_random_power_index(base_alice); 
print("Alice = ", Alice)
# Alice produces her public part Ka by calling the procedure nanapow() 
# that uses two parameters g and the power index Alice in order to 
# calculate g  to  the power of Alice
Ka = nanapow(g, Alice)
print("Ka = ", Ka)
print()

base_bob = 65
print("base_bob = ", base_bob)
# Bob gets a random power index
Bob = get_random_power_index(base_bob); 
print("Bob = ", Bob)
# Bob produces his public part Kb by calling the procedure nanapow() 
# that uses two parameters g and the power index Bob in order to 
# calculate g  to the power of Bob
Kb = nanapow(g, Bob)
print("Kb = ", Kb)
print()

# After receiving Kb, Alice computes the shared key Kab
Kab = nanapow(Kb, Alice)
print("Kab = ", Kab)
print()

# After receiving Ka, Bob computes the shared key Kba
Kba = nanapow(Ka, Bob)
print("Kba = ", Kba)
print()

print("The shared keys are the same: ", Kab == Kba)


initial seed:  0
lambda_bits =  256

p =  86844066927987146567678238756515930889952488499230423029593188005934847271147
log_2(p) =  256

Information about the finite field:  Finite Field of size 86844066927987146567678238756515930889952488499230423029593188005934847271147

aa1 =  32020853807654887644334957222306269398498608370866698553183453753385508071789
aa3 =  21006824032566107844541320689058119045737974467270935413688009986168488070826
aa4 =  78901668574955584791111916221370791050852265697346349228460454742358577919791
aa8 =  17514520152847203018279441394225361894014762419869662416889631828284746070238
bb1 =  21504609816581240890771596944610981247772392172380469982872574919132312646024
bb2 =  25223174955288415702340049551314795160644138670364684787245633274934513695975
bb5 =  51809049454113147465743186155015416837084822644379520043428264433478357770134
bb7 =  5452268907250569656859086919323520615353272469945652712304812007636937952392

E_Zero =  (51894466488054851875251155944967472

# Experiments with digital signatures 01

In [8]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 128
base = 257
GenKey_Signature01(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature01(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  255211775190703847597530955573826162347
log_2(p) =  128

Information about the finite field:  Finite Field of size 255211775190703847597530955573826162347

aa1 =  76196257025066871002891471619848650286
aa3 =  222398418873342498392506793217273317643
aa4 =  241322191999219874657044597267756778501
aa8 =  178720374391503507560897272085794393770
bb1 =  47717732877487248530043326597560953714
bb2 =  61733507447498311892063876037630235726
bb5 =  247856823686916680700123131434371243080
bb7 =  27228823810010487136687635290782267614

E_Zero =  (25958627203386039978290716591117649049, 211647247068710178442880791600586184585)
E_left_unit =  (174395086300108117424968928287131469871, 150946712368694434305233963210010197064)
E_minus_one =  (132733943297367810129143460468929990574, 17136006578022074982996664417336009759)

g =  (13486814435588948056647180649177578889, 232946345282352956340474695993456704856)

g_q =  (128137692344645973157570713635711768143, 200296064834253079673787

In [9]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 128
base = 17
GenKey_Signature01(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature01(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  255211775190703847597530955573826162347
log_2(p) =  128

Information about the finite field:  Finite Field of size 255211775190703847597530955573826162347

aa1 =  76196257025066871002891471619848650286
aa3 =  222398418873342498392506793217273317643
aa4 =  241322191999219874657044597267756778501
aa8 =  178720374391503507560897272085794393770
bb1 =  47717732877487248530043326597560953714
bb2 =  61733507447498311892063876037630235726
bb5 =  247856823686916680700123131434371243080
bb7 =  27228823810010487136687635290782267614

E_Zero =  (25958627203386039978290716591117649049, 211647247068710178442880791600586184585)
E_left_unit =  (174395086300108117424968928287131469871, 150946712368694434305233963210010197064)
E_minus_one =  (132733943297367810129143460468929990574, 17136006578022074982996664417336009759)

g =  (13486814435588948056647180649177578889, 232946345282352956340474695993456704856)

g_q =  (128137692344645973157570713635711768143, 200296064834253079673787

In [10]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 192
base = 257
GenKey_Signature01(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature01(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  4707826301540010572876842067405749812076766583348025913783
log_2(p) =  192

Information about the finite field:  Finite Field of size 4707826301540010572876842067405749812076766583348025913783

aa1 =  257405322579552167145111504184189350941678348657606363000
aa3 =  119532791761269064716056035369728457738265053964404214027
aa4 =  739437136086943117194679493097867279546663219437537419926
aa8 =  3296809007157619640119308770869998958606195085103878511269
bb1 =  1271238259397074869634643961725054659706124524993994915864
bb2 =  2806677724762809107594599379975401118512734879194091885646
bb5 =  4599163888442026094155062442360406257983778255665643166463
bb7 =  949464039987904334513186654173522437107012951521210331273

E_Zero =  (2674838327407532648726594500168436242421616629939967512484, 1660530161738285992366893199821047810162372675981459243468)
E_left_unit =  (3665473625056202058790254267668269814529715265285076709145, 2722100902305891965124393739497590144477489402438168

In [11]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 192
base = 17
GenKey_Signature01(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature01(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  4707826301540010572876842067405749812076766583348025913783
log_2(p) =  192

Information about the finite field:  Finite Field of size 4707826301540010572876842067405749812076766583348025913783

aa1 =  257405322579552167145111504184189350941678348657606363000
aa3 =  119532791761269064716056035369728457738265053964404214027
aa4 =  739437136086943117194679493097867279546663219437537419926
aa8 =  3296809007157619640119308770869998958606195085103878511269
bb1 =  1271238259397074869634643961725054659706124524993994915864
bb2 =  2806677724762809107594599379975401118512734879194091885646
bb5 =  4599163888442026094155062442360406257983778255665643166463
bb7 =  949464039987904334513186654173522437107012951521210331273

E_Zero =  (2674838327407532648726594500168436242421616629939967512484, 1660530161738285992366893199821047810162372675981459243468)
E_left_unit =  (3665473625056202058790254267668269814529715265285076709145, 2722100902305891965124393739497590144477489402438168

In [12]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 256
base = 257
GenKey_Signature01(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature01(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  86844066927987146567678238756515930889952488499230423029593188005934847271147
log_2(p) =  256

Information about the finite field:  Finite Field of size 86844066927987146567678238756515930889952488499230423029593188005934847271147

aa1 =  32020853807654887644334957222306269398498608370866698553183453753385508071789
aa3 =  21006824032566107844541320689058119045737974467270935413688009986168488070826
aa4 =  78901668574955584791111916221370791050852265697346349228460454742358577919791
aa8 =  17514520152847203018279441394225361894014762419869662416889631828284746070238
bb1 =  21504609816581240890771596944610981247772392172380469982872574919132312646024
bb2 =  25223174955288415702340049551314795160644138670364684787245633274934513695975
bb5 =  51809049454113147465743186155015416837084822644379520043428264433478357770134
bb7 =  5452268907250569656859086919323520615353272469945652712304812007636937952392

E_Zero =  (5189446648805485187525115594496747276449376761426439576

In [13]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 256
base = 17
GenKey_Signature01(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature01(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature01(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  86844066927987146567678238756515930889952488499230423029593188005934847271147
log_2(p) =  256

Information about the finite field:  Finite Field of size 86844066927987146567678238756515930889952488499230423029593188005934847271147

aa1 =  32020853807654887644334957222306269398498608370866698553183453753385508071789
aa3 =  21006824032566107844541320689058119045737974467270935413688009986168488070826
aa4 =  78901668574955584791111916221370791050852265697346349228460454742358577919791
aa8 =  17514520152847203018279441394225361894014762419869662416889631828284746070238
bb1 =  21504609816581240890771596944610981247772392172380469982872574919132312646024
bb2 =  25223174955288415702340049551314795160644138670364684787245633274934513695975
bb5 =  51809049454113147465743186155015416837084822644379520043428264433478357770134
bb7 =  5452268907250569656859086919323520615353272469945652712304812007636937952392

E_Zero =  (5189446648805485187525115594496747276449376761426439576

# Experiments with digital signatures 02

In [14]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 256
base = 257
GenKey_Signature02(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature02(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  86844066927987146567678238756515930889952488499230423029593188005934847271147
log_2(p) =  256

q =  43422033463993573283839119378257965444976244249615211514796594002967423635959
log_2(q) =  255

Information about the finite field:  Finite Field of size 86844066927987146567678238756515930889952488499230423029593188005934847271147

aa1 =  32020853807654887644334957222306269398498608370866698553183453753385508071789
aa3 =  21006824032566107844541320689058119045737974467270935413688009986168488070826
aa4 =  78901668574955584791111916221370791050852265697346349228460454742358577919791
aa8 =  17514520152847203018279441394225361894014762419869662416889631828284746070238
bb1 =  21504609816581240890771596944610981247772392172380469982872574919132312646024
bb2 =  25223174955288415702340049551314795160644138670364684787245633274934513695975
bb5 =  51809049454113147465743186155015416837084822644379520043428264433478357770134
bb7 =  54522689072505696568590869193235206153532724

In [15]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 256
base = 17
GenKey_Signature02(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature02(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  86844066927987146567678238756515930889952488499230423029593188005934847271147
log_2(p) =  256

q =  43422033463993573283839119378257965444976244249615211514796594002967423635959
log_2(q) =  255

Information about the finite field:  Finite Field of size 86844066927987146567678238756515930889952488499230423029593188005934847271147

aa1 =  32020853807654887644334957222306269398498608370866698553183453753385508071789
aa3 =  21006824032566107844541320689058119045737974467270935413688009986168488070826
aa4 =  78901668574955584791111916221370791050852265697346349228460454742358577919791
aa8 =  17514520152847203018279441394225361894014762419869662416889631828284746070238
bb1 =  21504609816581240890771596944610981247772392172380469982872574919132312646024
bb2 =  25223174955288415702340049551314795160644138670364684787245633274934513695975
bb5 =  51809049454113147465743186155015416837084822644379520043428264433478357770134
bb7 =  54522689072505696568590869193235206153532724

In [16]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 384
base = 257
GenKey_Signature02(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature02(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  29551504647295859409209280075107710353809804452849085000961220053184291328622907958560699691163686730604970992766987
log_2(p) =  384

q =  14775752323647929704604640037553855176904902226424542500480610026592145664311453979280349845581843365302485496383537
log_2(q) =  383

Information about the finite field:  Finite Field of size 29551504647295859409209280075107710353809804452849085000961220053184291328622907958560699691163686730604970992766987

aa1 =  7582394856132617620350301691287562681665178319115996216711488798994296321260008236339887422691230485947234341760057
aa3 =  20694405540067534286571407614852013949226916019336790190813868190978548966530225161940296844908863294027036466039051
aa4 =  28404507750296719480401111253101391650589452796426587072246545895132703299002258997178724078203659396904636222958928
aa8 =  5959882373095323157403881688469940215147421018937206763376731546015406306003074498066795537946055833375823442482254
bb1 =  9708054334335734886288470165

In [17]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 384
base = 17
GenKey_Signature02(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature02(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  29551504647295859409209280075107710353809804452849085000961220053184291328622907958560699691163686730604970992766987
log_2(p) =  384

q =  14775752323647929704604640037553855176904902226424542500480610026592145664311453979280349845581843365302485496383537
log_2(q) =  383

Information about the finite field:  Finite Field of size 29551504647295859409209280075107710353809804452849085000961220053184291328622907958560699691163686730604970992766987

aa1 =  7582394856132617620350301691287562681665178319115996216711488798994296321260008236339887422691230485947234341760057
aa3 =  20694405540067534286571407614852013949226916019336790190813868190978548966530225161940296844908863294027036466039051
aa4 =  28404507750296719480401111253101391650589452796426587072246545895132703299002258997178724078203659396904636222958928
aa8 =  5959882373095323157403881688469940215147421018937206763376731546015406306003074498066795537946055833375823442482254
bb1 =  9708054334335734886288470165

In [18]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 512
base = 257
GenKey_Signature02(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature02(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))

initial seed:  0
p =  10055855947456947824680518748654384595609524365444295033292671082791323022555160232601405723625177570767523893639864538140315412108959927459825236754597199
log_2(p) =  512

q =  5027927973728473912340259374327192297804762182722147516646335541395661511277580116300702861812588785383761946819932269070157706054479963729912618377298809
log_2(q) =  511

Information about the finite field:  Finite Field of size 10055855947456947824680518748654384595609524365444295033292671082791323022555160232601405723625177570767523893639864538140315412108959927459825236754597199

aa1 =  5083521238499181616107103435540117463555863868499725717345286618378307579761303457915289965842449989320867780251139340372938505409275428692727806563961170
aa3 =  2432424042971493215259752716409360521732123369897594983538095158510825326037876699129932315236342603157409792578609891994907734721051329531278165610547467
aa4 =  2622432305419929059970488180595521576690001852579062212738857317591449896662904619

In [76]:
# The results of this code are produced in SageMath 9.2
# To ensure the reproducibility of the results, set a fixed initial seed
set_random_seed(0)
random.seed(0, version=2)
print("initial seed: ", initial_seed())

lambda_bits = 512
base = 17
GenKey_Signature02(lambda_bits, base)

print()
Message = "Try to sign this message!"
print("Signing the message: ", Message)
signature = Sign_Signature02(PrivateKey, Message, base, lambda_bits)
print("signature = ", signature)
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))
print()
# Corrupt the Message
Message = "Try to sign this message"
print("An attampt to verify a signature of a corrupted message")
print("Verify: ", Verify_Signature02(PublicKey, Message, signature, base, lambda_bits))

p =  10055855947456947824680518748654384595609524365444295033292671082791323022555160232601405723625177570767523893639864538140315412108959927459825236754597199
log_2(p) =  512

q =  5027927973728473912340259374327192297804762182722147516646335541395661511277580116300702861812588785383761946819932269070157706054479963729912618377298809
log_2(q) =  511

Information about the finite field:  Finite Field of size 10055855947456947824680518748654384595609524365444295033292671082791323022555160232601405723625177570767523893639864538140315412108959927459825236754597199

aa1 =  8393591886190197485847402848711640133588380435307610336904487690048049556248693181317648695833075233082558245368227166750509251971063164118784581543020449
aa3 =  1731553306913410855827384355597123889285738137106118908367456612755620222478685084904487362926038412234565895435680359512401176170207761669433168955763067
aa4 =  441461756384708867091872716646130896617536522759534668726880234160614621296929318306936020771824796